In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        if self.name =="phone":
            for word in sentence.split('_'):
                self.addWord(word)
        else:
            for word in list(sentence):
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/home/zedroman/Documents/Sonia_Data/eng_phonetics/train.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split(' ')[0:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 76

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
#     print(pairs)
    print("Read %s sentence pairs" % len(pairs))
#     pairs = filterPairs(pairs)
#     print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('words', 'phone', True)
print(random.choice(pairs))



Reading lines...
Read 83194 sentence pairs
Counting words...
Counted words:
words 30
phone 41
['DENMAN', 'D_EH_N_M_AH_N']


In [7]:
print(input_lang)
print(input_lang.word2index)
print(output_lang.word2index)

{'L': 2, 'E': 3, 'M': 4, 'I': 5, 'U': 6, 'X': 7, 'N': 8, 'D': 9, 'G': 10, 'S': 11, 'T': 12, 'R': 13, 'P': 14, 'K': 15, 'C': 16, 'O': 17, 'F': 18, 'A': 19, 'B': 20, 'H': 21, 'V': 22, 'Y': 23, 'W': 24, 'J': 25, "'": 26, 'Q': 27, 'Z': 28, '-': 29}
{'L': 2, 'AH': 3, 'M': 4, 'Y': 5, 'UW': 6, 'AY': 7, 'N': 8, 'D': 9, 'IH': 10, 'NG': 11, 'S': 12, 'T': 13, 'R': 14, 'P': 15, 'K': 16, 'EH': 17, 'AA': 18, 'F': 19, 'ER': 20, 'EY': 21, 'AE': 22, 'Z': 23, 'G': 24, 'B': 25, 'SH': 26, 'V': 27, 'OW': 28, 'AO': 29, 'IY': 30, 'W': 31, 'HH': 32, 'JH': 33, 'CH': 34, 'TH': 35, 'AW': 36, 'OY': 37, 'UH': 38, 'ZH': 39, 'DH': 40}


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size * 2
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
def indexesFromSentence(lang, sentence, out, back = False):
    if not out:
        if back:
            return list(reversed([lang.word2index[word] for word in list(sentence)]))
        return [lang.word2index[word] for word in list(sentence)]
    else:
        return [lang.word2index[word] for word in sentence.split('_')]


def tensorFromSentence(lang, sentence, out, back = False):
    indexes = indexesFromSentence(lang, sentence, out, back)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0], 0)
    target_tensor = tensorFromSentence(output_lang, pair[1], 1)
    back_tensor = tensorFromSentence(input_lang, pair[0], 0, back=True)
    return (input_tensor, target_tensor, back_tensor)

In [32]:
teacher_forcing_ratio = 1


def train(input_tensor, input_tensor_back, target_tensor, encoder, encoder_back, decoder, encoder_optimizer,encoder_back_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_back_hidden = encoder_back.initHidden()

    encoder_optimizer.zero_grad()
    encoder_back_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    encoder_back_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        encoder_back_output, encoder_back_hidden = encoder_back(
            input_tensor_back[ei], encoder_back_hidden)
        encoder_back_outputs[ei] = encoder_back_output[0, 0]

    encoder_hidden = torch.cat((encoder_hidden, encoder_back_hidden), 2)
    encoder_outputs = torch.cat((encoder_outputs, encoder_back_outputs), 1)
    
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            print(decoder_output.shape)
            print(target_tensor[di].shape)
            print()
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    encoder_back_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [23]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [24]:
training_pairs = [tensorsFromPair(pairs[i])
                  for i in range(len(pairs))]

def trainIters(encoder, encoder_back, decoder, n_iters, encoder_optimizer, encoder_back_optimizer, decoder_optimizer, print_every=1000, plot_every=100, learning_rate=0.001, epoch=0):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = random.choice(training_pairs)
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        input_tensor_back = training_pair[2]

        loss = train(input_tensor, input_tensor_back, target_tensor, encoder, encoder_back,
                     decoder, encoder_optimizer,encoder_back_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
#             torch.save(encoder1, '/home/zedroman/Documents/Sonia_Data/test_models2/encoder_epoch{}_{}'.format(epoch, iter))
#             torch.save(attn_decoder1, '/home/zedroman/Documents/Sonia_Data/test_models2/decoder_epoch{}_{}'.format(epoch, iter))
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

#         if iter % plot_every == 0:
#             plot_loss_avg = plot_loss_total / plot_every
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0

#     showPlot(plot_losses)

In [25]:
def evaluate(encoder, encoder_back, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, 0, 0)
        input_tensor_back = tensorFromSentence(input_lang, sentence, 0, 1)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_back_hidden = encoder_back.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        encoder_back_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            encoder_back_output, encoder_back_hidden = encoder_back(
                input_tensor_back[ei], encoder_back_hidden)
            encoder_back_outputs[ei] = encoder_back_output[0, 0]

        encoder_hidden = torch.cat((encoder_hidden, encoder_back_hidden), 2)
        encoder_outputs = torch.cat((encoder_outputs, encoder_back_outputs), 1)

            

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
#                 decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [26]:
def evaluateRandomly(encoder,encoder_back, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder,encoder_back, decoder, pair[0])
        output_sentence = '_'.join(output_words)
        print('<', output_sentence)
        print('')

In [27]:
def checkAccuracy(encoder,encoder_back, decoder, choose = 5000):
    good = 0
    for i in range(choose):
        pair = random.choice(pairs)
        output_words, attentions = evaluate(encoder,encoder_back, decoder, pair[0])
        output_sentence = '_'.join(output_words)
        if (output_sentence == pair[1]):
            good+=1
    print(good / choose)

In [28]:
hidden_size = 200
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder_back = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
epo = 6
# encoder = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/encoder_epoch{}'.format(epo)).to(device)
# encoder_back = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/encoder_back_epoch{}'.format(epo)).to(device)
# attn_decoder = torch.load( '/home/zedroman/Documents/Sonia_Data/test_models4/decoder_epoch{}'.format(epo)).to(device)
# encoder1.eval()
# attn_decoder1.eval()

In [29]:
print(device)
print(torch.cuda.is_available())

cpu
False


In [30]:
learning_rate = 0.0001
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate, momentum=0.9)
encoder_back_optimizer = optim.SGD(encoder_back.parameters(), lr=learning_rate, momentum=0.9)
decoder_optimizer = optim.SGD(attn_decoder.parameters() , lr=learning_rate, momentum=0.9)

In [33]:

for epoch in range(6, 100):
#     83194
    trainIters(encoder, encoder_back, attn_decoder, 83194, encoder_optimizer, encoder_back_optimizer, decoder_optimizer, print_every=200, epoch=epoch, learning_rate=learning_rate)
    torch.save(encoder, '/home/zedroman/Documents/Sonia_Data/test_models4/encoder_epoch{}'.format(epoch))
    torch.save(encoder_back, '/home/zedroman/Documents/Sonia_Data/test_models4/encoder_back_epoch{}'.format(epoch))
    torch.save(attn_decoder, '/home/zedroman/Documents/Sonia_Data/test_models4/decoder_epoch{}'.format(epoch))

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

torch.Size([1, 41])
torch.Size([1])

t

KeyboardInterrupt: 

In [ ]:
epo = 1
encoder = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/encoder_epoch{}'.format(epo))
attn_decoder = torch.load( '/home/zedroman/Documents/Sonia_Data/test_models4/decoder_epoch{}'.format(epo))
encoder.eval()
attn_decoder.eval()

In [ ]:
evaluateRandomly(encoder, encoder_back, attn_decoder)
checkAccuracy(encoder,encoder_back, attn_decoder)
# evaluateRandomly(encoder1, attn_decoder1)
# checkAccuracy(encoder1, attn_decoder1)

In [ ]:
import pandas as pd
txt = pd.read_csv('/home/zedroman/Documents/Sonia_Data/eng_phonetics/test(1).csv').as_matrix()
index = txt[:, 0]
test = txt[:,1]
ans = []
for word in test:
    output_words, attentions = evaluate(encoder, encoder_back, attn_decoder, word)
    ans.append('_'.join(output_words))
ans=np.array(ans)
ret = pd.DataFrame({'Id':index, 'Transcription':ans})
ret.to_csv("/home/zedroman/Documents/Sonia_Data/eng_phonetics/ans.csv", index=False)